In [1]:
import pprint
from jsonschema.validators import Draft7Validator
import json

"""
Open a Mosdex file and validate it against the schema
"""

MOSDEX_SCHEMA_FILE = "MOSDEXSchemaV2-1.json"
MOSDEX_FILE = "sailco_2-1.json"

with open(MOSDEX_SCHEMA_FILE, "r") as f:
    schema = json.load(f)
validator = Draft7Validator(schema)

with open(MOSDEX_FILE, "r") as f:
    mosdex = json.load(f)

if not validator.is_valid(mosdex):
    print(f"File {MOSDEX_FILE} is not a valid Mosdex file.")
    pp = pprint.PrettyPrinter(indent=4)
    for error in sorted(validator.iter_errors(mosdex), key=str):
        print()
        pp.pprint(error.message)
else:
    print(f"File {MOSDEX_FILE} is a valid instance of schema {MOSDEX_SCHEMA_FILE}.")


File sailco_2-1.json is a valid instance of schema MOSDEXSchemaV2-1.json.


In [2]:
"""
Print some features of the MOSDEX_FILE
"""

print(f"The syntax of {MOSDEX_FILE} is {mosdex['SYNTAX']}")
for module in mosdex['MODULES']:
    print(f"\nModule {module['NAME']} is a {module['KIND']}")
    

The syntax of sailco_2-1.json is MOSDEX/MOSDEX v2-0/MOSDEXSchemaV2-0.json

Module sailco is a MODEL


In [3]:
# Get reference to a module that has KIND == MODEL
model = {}
for module in mosdex['MODULES']:
    if module['KIND'] == 'MODEL':
        model = module
        break

print(f"Got handle to MODEL: {model['NAME']}")
print(f"The sections of the model are {list(model.keys())}")
print(f"\tNAME: {model['NAME']}")
print(f"\tCLASS: {model['CLASS']}")
print(f"\tKIND: {model['KIND']}")
print(f"\tTABLES: there are {len(model['TABLES'])} tables:")
for table in model['TABLES']:
    print(f"\t\t{table['NAME']:10s} \t class/kind: {table['CLASS']}/{table['KIND']}")

Got handle to MODEL: sailco
The sections of the model are ['NAME', 'CLASS', 'KIND', 'HEADING', 'TABLES']
	NAME: sailco
	CLASS: MODULE
	KIND: MODEL
	TABLES: there are 13 tables:
		demands    	 class/kind: DATA/INPUT
		parameters 	 class/kind: DATA/INPUT
		regular    	 class/kind: VARIABLE/CONTINUOUS
		extra      	 class/kind: VARIABLE/CONTINUOUS
		inventory  	 class/kind: VARIABLE/CONTINUOUS
		cost       	 class/kind: VARIABLE/CONTINUOUS
		totalCost  	 class/kind: VARIABLE/CONTINUOUS
		ctCapacity 	 class/kind: CONSTRAINT/LINEAR
		ctBoat     	 class/kind: CONSTRAINT/LINEAR
		ctBoatMatrix 	 class/kind: MATRIX/LINEAR
		ctCost     	 class/kind: CONSTRAINT/LINEAR
		ctCostMatrix 	 class/kind: MATRIX/LINEAR
		production 	 class/kind: DATA/OUTPUT


In [4]:
"""
Initialize the database engine
"""
from sqlalchemy import create_engine, Integer, String, Double, ForeignKey

engine = create_engine('sqlite:///:memory:')

from sqlalchemy.orm import declarative_base, mapped_column

Base = declarative_base()

# Declare each table
for table_ in model['TABLES']:
    table_name = table_['NAME']
    table_class = table_['CLASS']
    table_kind = table_['KIND']
    table_schema = table_['SCHEMA']
    
    # Dictionary for database info
    db_table: dict = {} 
    
    # Class name for rows
    db_table['class_name'] = table_['CLASS'] + '_' + model['NAME'] + '_' + table_['NAME']
    
    # Load the table_attr dictionary
    db_table['attr'] = { '__tablename__': table_name,
                      'id': mapped_column(Integer, primary_key=True)}
    
    # Apply the SCHEMA to the table columns
    for name, kind in zip(table_schema['NAME'], table_schema['KIND']):
        
        # Column type
        if kind == 'INTEGER':
            type_col = Integer
        elif kind == 'DOUBLE' or kind == 'DOUBLE_FUNCTION':
            type_col = Double
        elif kind == 'STRING':
            type_col = String
        else:
            print(f"Error, type {kind} is not supported")
            break
        
        # Add the column to the table_attr dictionary
        # - set primary_key flag 
        # - set Foreign Key relationship 
        if 'KEYS' in table_schema and name in table_schema['KEYS']:
            # Primary Key
            db_table['attr'][name] = mapped_column(type_col, primary_key=True)
        if 'FOREIGN_KEYS' in table_schema and name in table_schema['FOREIGN_KEYS']:
            # Foreign Key
            f_key = table_schema['FOREIGN_KEYS'][name]
            db_table['attr'][name] = mapped_column(type_col, ForeignKey(f_key))
        else:
            db_table['attr'][name] = mapped_column(type_col)
    
    # Declarative instantiation of the table     
    db_table['instance'] = type(db_table['class_name'], (Base,), db_table['attr'])
    
    # Add the db_table dictionary to the TABLE dict
    table_['db_table'] = db_table
    
    
Base.metadata.create_all(engine)
        
print(f"Database tables created")
for table in Base.metadata.tables.keys():
    print(f"\t{table}")
    print(f"\t\t{Base.metadata.tables[table].columns.keys()}")
    

Database tables created
	demands
		['id', 'period', 'ancestor', 'demand']
	parameters
		['id', 'regularCost', 'extraCost', 'capacity', 'initialInventory', 'inventoryCost']
	regular
		['id', 'name', 'period', 'colName', 'lowerBound', 'upperBound', 'primalValue']
	extra
		['id', 'name', 'period', 'colName', 'lowerBound', 'upperBound', 'primalValue']
	inventory
		['id', 'name', 'period', 'colName', 'lowerBound', 'upperBound', 'primalValue']
	cost
		['id', 'name', 'period', 'colName', 'lowerBound', 'upperBound', 'primalValue']
	totalCost
		['id', 'colName', 'lowerBound', 'upperBound', 'primalValue']
	ctCapacity
		['id', 'name', 'period', 'rowName', 'lowerBound', 'upperBound', 'dualValue']
	ctBoat
		['id', 'name', 'period', 'rowName', 'sense', 'RHS', 'dualValue']
	ctBoatMatrix
		['id', 'period', 'rowName', 'regularName', 'regularCoefficient', 'extraName', 'extraCoefficient', 'inventoryName', 'inventoryCoefficient', 'laggedInventoryName', 'laggedInventoryCoefficient']
	ctCost
		['id', 'name'

In [5]:
from sqlalchemy import text, Table
from sqlalchemy.orm import Session
from tabulate import tabulate

"""
Load the INPUT data
"""
import pandas as pd
import numpy as np

for table_ in model['TABLES']:
    if table_['KIND'] == 'INPUT':
        if "INSTANCE" in table_ and table_['CLASS'] == "DATA":
            # Get column names 
            col_names = table_['SCHEMA']['NAME']
            
            # Create a dataframe from the INSTANCE arrays
            data_df = pd.DataFrame(np.vstack(table_['INSTANCE']), columns=col_names)
            print("\n>>DATAFRAME<< ")
            print(data_df.head())
    
            # Push the dataframe to the table
            with Session(engine) as session, session.begin():
                data_df.to_sql(name=table_['NAME'], con=session.connection(),
                                       if_exists='append', index=False)
                session.flush()
                stmt = "select * from " + table_['NAME']
                result = session.execute(text(stmt))
                
                print("\n>>TABLE<<")
                t = Table(table_['NAME'], Base.metadata, autoload=True, autoload_with=engine)
                print(tabulate(result, headers=t.columns.keys(), tablefmt="fancy_grid"))
        else:
            # Load in another way
            pass
    else:
        # Not INPUT 
        pass



>>DATAFRAME<< 
   period  ancestor  demand
0     1.0       0.0    40.0
1     2.0       1.0    60.0
2     3.0       2.0    75.0
3     4.0       3.0    25.0

>>TABLE<<
╒══════╤══════════╤════════════╤══════════╕
│   id │   period │   ancestor │   demand │
╞══════╪══════════╪════════════╪══════════╡
│    1 │        1 │          0 │       40 │
├──────┼──────────┼────────────┼──────────┤
│    2 │        2 │          1 │       60 │
├──────┼──────────┼────────────┼──────────┤
│    3 │        3 │          2 │       75 │
├──────┼──────────┼────────────┼──────────┤
│    4 │        4 │          3 │       25 │
╘══════╧══════════╧════════════╧══════════╛

>>DATAFRAME<< 
   regularCost  extraCost  capacity  initialInventory  inventoryCost
0        400.0      450.0      40.0              10.0           20.0

>>TABLE<<
╒══════╤═══════════════╤═════════════╤════════════╤════════════════════╤═════════════════╕
│   id │   regularCost │   extraCost │   capacity │   initialInventory │   inventoryCost │
╞═

In [6]:
from sqlalchemy import Table

"""
Process a VARIABLE
"""

# demands DATA table
demands = Table('demands', Base.metadata, autoload_with=engine)

# regular VARIABLE table
regular = Table('regular', Base.metadata, autoload_with=engine)

# Append rows to 'regular' 
with Session(engine) as session, session.begin():
    stmt = text("INSERT INTO regular (name, period, colName, lowerBound, upperBound, primalValue) "
                "SELECT "
                "'regular', "
                "period , "
                "CONCAT('regular','_', period), "
                "0.0 , "
                "'Infinity', "
                "NULL "
                "FROM demands")
    result = session.execute(stmt)
    session.flush()
    
    # Print table
    rows = session.execute(regular.select())
    print(tabulate(rows, headers=regular.columns.keys(), tablefmt="fancy_grid"))
        



╒══════╤═════════╤══════════╤═══════════╤══════════════╤══════════════╤═══════════════╕
│   id │ name    │   period │ colName   │   lowerBound │ upperBound   │ primalValue   │
╞══════╪═════════╪══════════╪═══════════╪══════════════╪══════════════╪═══════════════╡
│    1 │ regular │        1 │ regular_1 │            0 │ Infinity     │               │
├──────┼─────────┼──────────┼───────────┼──────────────┼──────────────┼───────────────┤
│    2 │ regular │        2 │ regular_2 │            0 │ Infinity     │               │
├──────┼─────────┼──────────┼───────────┼──────────────┼──────────────┼───────────────┤
│    3 │ regular │        3 │ regular_3 │            0 │ Infinity     │               │
├──────┼─────────┼──────────┼───────────┼──────────────┼──────────────┼───────────────┤
│    4 │ regular │        4 │ regular_4 │            0 │ Infinity     │               │
╘══════╧═════════╧══════════╧═══════════╧══════════════╧══════════════╧═══════════════╛


In [7]:
"""
Process a CONSTRAINT
"""

# Get ctBoat table
ctBoat = Table('ctBoat', Base.metadata, autoload_with=engine)

# Generate ctBoat table with initialInventory as RHS in period 1
with Session(engine) as session, session.begin():
    stmt = text("INSERT INTO ctBoat (name, period, rowName, sense, RHS, dualValue )"
                "SELECT "
                "'ctBoat' , "
                "period , "
                "CONCAT('ctBoat','_', period), "
                "'==' AS sense,"
                "CASE "
                "WHEN period > 1 THEN 0 "
                "WHEN period ==1 THEN initialInventory "
                "END AS RHS, "
                "NULL AS dualValue "
                "FROM demands, parameters")
    session.execute(stmt)
    session.flush()
    # Print table
    rows = session.execute(ctBoat.select())
    print(tabulate(rows, headers=ctBoat.columns.keys(), tablefmt="fancy_grid"))
    

╒══════╤════════╤══════════╤═══════════╤═════════╤═══════╤═════════════╕
│   id │ name   │   period │ rowName   │ sense   │   RHS │ dualValue   │
╞══════╪════════╪══════════╪═══════════╪═════════╪═══════╪═════════════╡
│    1 │ ctBoat │        1 │ ctBoat_1  │ ==      │    10 │             │
├──────┼────────┼──────────┼───────────┼─────────┼───────┼─────────────┤
│    2 │ ctBoat │        2 │ ctBoat_2  │ ==      │     0 │             │
├──────┼────────┼──────────┼───────────┼─────────┼───────┼─────────────┤
│    3 │ ctBoat │        3 │ ctBoat_3  │ ==      │     0 │             │
├──────┼────────┼──────────┼───────────┼─────────┼───────┼─────────────┤
│    4 │ ctBoat │        4 │ ctBoat_4  │ ==      │     0 │             │
╘══════╧════════╧══════════╧═══════════╧═════════╧═══════╧═════════════╛


In [8]:
"""
Process a MATRIX
"""

# Matrix table for ctBoat

ctBoatMatrix = Table('ctBoatMatrix', Base.metadata, autoload_with=engine)

# Generate matrix for Inventory Balance equation
with Session(engine) as session, session.begin():
    stmt = text(
        "INSERT INTO ctBoatMatrix (period, rowName, regularName, regularCoefficient,"
        "extraName, extraCoefficient, inventoryName, inventoryCoefficient, laggedInventoryName, laggedInventoryCoefficient) "
        "SELECT "
                "period , "
                "CONCAT('ctBoat', '_', period) , "
                "CONCAT('regular','_', period), "
                "1.0 , "
                
                "CONCAT('extra','_', period) , "
                "1.0, "
                
                "CONCAT('inventory','_', period) , "
                "-1.0 , "
                
                "CASE "
                "WHEN period > 1 THEN CONCAT('inventory','_', ancestor) "
                "ELSE NULL END , "
                "CASE "
                "WHEN period > 1 THEN 1.0 "
                "ELSE NULL END  "
                
                "FROM demands")
    session.execute(stmt)
    session.flush()
    # Print table
    rows = session.execute(ctBoatMatrix.select())
    print(tabulate(rows, headers=ctBoatMatrix.columns.keys(), tablefmt="fancy_grid"))
    

╒══════╤══════════╤═══════════╤═══════════════╤══════════════════════╤═════════════╤════════════════════╤═════════════════╤════════════════════════╤═══════════════════════╤══════════════════════════════╕
│   id │   period │ rowName   │ regularName   │   regularCoefficient │ extraName   │   extraCoefficient │ inventoryName   │   inventoryCoefficient │ laggedInventoryName   │   laggedInventoryCoefficient │
╞══════╪══════════╪═══════════╪═══════════════╪══════════════════════╪═════════════╪════════════════════╪═════════════════╪════════════════════════╪═══════════════════════╪══════════════════════════════╡
│    1 │        1 │ ctBoat_1  │ regular_1     │                    1 │ extra_1     │                  1 │ inventory_1     │                     -1 │                       │                              │
├──────┼──────────┼───────────┼───────────────┼──────────────────────┼─────────────┼────────────────────┼─────────────────┼────────────────────────┼───────────────────────┼────────────